# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

In [1]:
import re
import pandas as pd

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [2]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

[print(f"{k:>10} = '{v}'") for k, v in obj.items()]

 home_page = 'https://github.com/pypa/sampleproject'
  keywords = 'sample setuptools development'
   license = 'MIT'


[None, None, None]

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [3]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
sum([re.findall(r"\d{2}-\d{1}", row) for row in obj], [])

['19-1', '20-4', '20-3']

3. Разбейте текст формулировки задачи 2 на слова.

In [4]:
text = "2. Написать регулярное выражение,которое позволит найти номера групп студентов."
#sum([re.findall(r"\b[а-яА-ЯёЁ]+\b", row.strip()) for row in text.split()], [])

#[word for sublist in [re.findall(r"\b[а-яА-ЯёЁ]+\b", row) for row in text.split()] for word in sublist]

bank = [re.findall(r"\b[а-яА-ЯёЁ]+\b", row) for row in text.split()]
[word for sublist in bank for word in sublist]

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [6]:
recipes = pd.read_csv("../data sources/recipes_sample.csv")
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [7]:
five_r = recipes.sample(n=5)

max_len_id = len(str(max(five_r.id))) + 2
max_len_minutes = len(str(max(five_r.minutes))) + 2

len_id = [len('id') + 2, max_len_id][len('id') < max_len_id]
len_minutes = [len('minutes') + 2, max_len_minutes][max_len_minutes > len('minutes')]


print(f"|{'id':^{len_id}}|{'minutes':^{len_minutes}}|")
[print(f"|{v1:^{len_id}}|{v2:^{len_minutes}}|") for v1, v2 in zip(five_r.id, five_r.minutes)]

|   id   | minutes |
| 370684 |   30    |
| 204642 |   15    |
| 29870  |   70    |
| 429559 |    5    |
| 484315 |  1450   |


[None, None, None, None, None]

2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [8]:
from bs4 import BeautifulSoup

content = open("../data sources/steps_sample.xml","r").read()
steps = BeautifulSoup(content,'xml')
#steps

In [9]:
from collections import defaultdict

bank = defaultdict(list)

for recipe in steps.find_all('recipe'):
    recipe_id = int(recipe.find("id").get_text())
    for step in recipe.find_all("step"):
        step_text = step.get_text()
        bank[recipe_id].append(step_text)

#bank

In [10]:
def show_info(recipe_id, recipes, bank):
    name, rid, minutes, author_id, *other = recipes[recipes.id == recipe_id].values[0]
    steps = bank[recipe_id]
    print(name, end='\n\n')
    [print(f"{k}. {v}") for k, v in enumerate(steps, 1)]
    print("----------")
    print(f"Автор: {author_id}\nСреднее время приготовления: {minutes} минут")

show_info(170895, recipes, bank)

leeks and parsnips  sauteed or creamed

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an additi

In [11]:
show_info(44123, recipes, bank)

george s at the cove  black bean soup

1. in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes
2. add thyme , oregano and basil
3. saute 5 minutes more
4. add wine and deglaze pan
5. add hot chicken stock and reduce by one-third
6. add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets
7. simmer 5 minutes
8. add cream , simmer 5 minutes more and season to taste
9. drop in remaining butter , piece by piece , stirring until melted and serve immediately
10. smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare
11. chef meskan uses applewood chips and does not allow the grill to become too hot
----------
Автор: 35193
Среднее время приготовления: 90 минут


In [ ]:
"""
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)
"""

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [12]:
step25082 = bank[25082]
result = [(i, re.findall(r"[-0-9]+ minute[s]?|[0-9]+ hour[s]?", row)) for i, row in enumerate(step25082, 1)]

[print(f"Найдено в шаге {row[0]}: {row[1]}") for row in result if row[1]]

Найдено в шаге 6: ['20 minutes']
Найдено в шаге 8: ['10 minutes']
Найдено в шаге 10: ['2 hours']
Найдено в шаге 14: ['10 minutes']
Найдено в шаге 17: ['20 minutes', '20-30 minutes']


[None, None, None, None, None]

In [13]:
# Проверка
step25082[5] # Найдено в шаге 6: ['20 minutes']

'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board'

In [14]:
# Проверка
step25082[16]# Найдено в шаге 17: ['20 minutes', '20-30 minutes']

'bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom'

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [15]:
pattern = re.compile(r"^this.*,\s?but")

text ='this really, .&? really cool, but i think'

re.findall(pattern, text)

['this really, .&? really cool, but']

In [16]:
test ='this really, i think,but really cool '

re.findall(pattern, test)

['this really, i think,but']

In [17]:
clear_recipes = recipes.dropna()
clear_recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
6,open sesame noodles,107229,28,173674,2004-12-30,8.0,this is a very versatile and widely enjoyed pa...,12.0
8,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,12.0,this is the recipe that we use at my school ca...,11.0
15,better than tofu cheesecake,200148,90,261027,2006-12-11,18.0,"this is, again, from vegweb.com. this got abso...",13.0
16,burek or feta cheese phyllo pie,310570,65,676820,2008-06-24,38.0,"ok, there are different version of burek (some...",6.0
21,kelly s creamy cheddar pea salad,125195,20,113941,2005-06-09,5.0,"i'm not a big fan of peas, but like them in th...",10.0


In [18]:
mask = clear_recipes.description.str.match(pattern)
print(f"Количество таких рецептов = {mask.sum()}")

Количество таких рецептов = 370


In [19]:
pd.set_option('display.max_colwidth', None)

clear_recipes[mask][:3].description

202                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 this is the simplest toffee recipe ever.  i love it because it only takes 4 ingredients that i always have around the house.  follow a few easy steps and voila--instant gratification!  this recipe makes great holiday gifts for the neighbors (they'll think you slaved over a hot stove all day, but the truth will be our little secret!).
271                                                                                                                                                 

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [20]:
steps72367 = bank[72367]
steps72367

['mix butter , flour , 1 / 3 c',
 'sugar and 1-1 / 4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1 / 4 c',
 'sugar , eggs and 1 / 2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1 / 3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

In [21]:
pattern = r"\s/\s"
repl = r"/"
result = [re.sub(pattern, repl, row) for row in steps72367]
result

['mix butter , flour , 1/3 c',
 'sugar and 1-1/4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1/4 c',
 'sugar , eggs and 1/2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1/3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [59]:
# import nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cvrsd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [22]:
from nltk import word_tokenize

nltk_words = []
    
for k, v in bank.items():
    raw_words = list(filter(str.isalpha, word_tokenize(' '.join(v).lower())))
    nltk_words.extend(raw_words)


print(f"\n{'Количество всех слов:':>30} {len(nltk_words)}\n{'Количество уникальных слов:':>30} {len(set(nltk_words))}")


         Количество всех слов: 2734377
   Количество уникальных слов: 14926


## 2 реализация

In [23]:
all_words = []
    
for k, v in bank.items():
    c_steps = re.findall(r"\b[a-zA-Zа-яА-ЯёЁ]+\b", ' '.join(v).lower()) #очищенные шаги
    all_words.extend(c_steps)

print(f"\n{'Количество всех слов:':>30} {len(all_words)}\n{'Количество уникальных слов:':>30} {len(set(all_words))}")
    



         Количество всех слов: 2765741
   Количество уникальных слов: 15114


# странно

In [24]:
print(set(all_words) - set(nltk_words))


{'preparethe', 'mono', 'didn', 'vegans', 'drainbroccoli', 'indo', 'atar', 'orangebutterscotch', 'loup', 'hign', 'ince', 'reflexes', 'tupled', 'lll', 'turf', 'sinus', 'rated', 'wanna', 'hearted', 'ventilated', 'gonna', 'mouthed', 'marchandde', 'afteryou', 'bulls', 'tayavon', 'lemme', 'fourts', 'peasy', 'wasn', 'stic', 'crafts', 'andbread', 'heatdon', 'hasn', 'doh', 'addonion', 'hydrate', 'cgarlic', 'disaronnomaple', 'azur', 'oeuvres', 'sixteenth', 'pooh', 'greedy', 'barb', 'ivory', 'chunless', 'ceam', 'tel', 'equatoriale', 'haves', 'clearing', 'scary', 'straps', 'bill', 'isn', 'removechicken', 'friedonions', 'socket', 'pom', 'haven', 'addthe', 'bemi', 'assem', 'wah', 'borne', 'placecooked', 'butdon', 'nu', 'botomed', 'toprepare', 'pe', 'shocks', 'germain', 'caffeine', 'str', 'framed', 'ey', 'surf', 'gots', 'tic', 'addover', 'cin', 'parmesano', 'cinnamonraisin', 'streaky', 'fijian', 'chocoalte', 'lantern', 'crossed', 'anti', 'sans', 'chi', 'mufa', 'reux', 'wishers', 'fla', 'cum', 'hihgh'

In [25]:
print(set(nltk_words) - set(all_words))

{'gon', 'lem', 'wan', 'butdo', 'mus', 'wo', 'heatdo'}


## 3 реализация

In [26]:
import re
from collections import Counter

#bank.values()
row = ' '.join([word.lower() for row in bank.values() for word in row]) #одна огромная строка из всех шагов
pattern = r"\b[a-zA-Zа-яА-ЯёЁ]+\b"
cwords = re.findall(pattern, row) #очищенные от лишнего слова
unique_words = Counter(cwords) #словарь уникальных слов
unique_words.most_common(5)

[('and', 147015), ('the', 138058), ('in', 77528), ('a', 75336), ('to', 74875)]

In [27]:
print(f"Количество уникальных слов: {len(unique_words)}\nКоличество всех слов: {sum(unique_words.values())}")

Количество уникальных слов: 15114
Количество всех слов: 2765741


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [28]:
from nltk import sent_tokenize

description = recipes.description.astype(str)

all_sentences = [sent_tokenize(d) for d in description]
most_long = sorted(all_sentences, key=lambda x: len(x), reverse=True)[:5]


len(most_long[0]), len(most_long[-1])

(76, 23)

In [33]:
most_long

[['this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.',
  'it makes a delicious filling also between the layers of cakes and under fondant icing.',
  'you can make roses but it takes 3 or more days to dry them depending on the humidity.',
  'there are many versions of “buttercream” icing.',
  'some are made with eggs and all butter.',
  'some varieties, you have to cook your sugar to a softball stage.',
  'others are 100% shortening or a combination of shortening and butter.',
  'each decorator has his or her favorite.',
  'i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch.',
  'but….',
  'i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.',
  'it melts!',
  'a greasy puddle of melted icing on a cake plate is not something i want to look at or eat.',
  'yo

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.



In [73]:
#nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cvrsd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [34]:
from nltk import pos_tag, word_tokenize

check = "I  omitted the raspberries and added strawberries instead"

words = word_tokenize(check)
word_pos = pos_tag(words)
[print(f"{k:^15} - {v}") for k, v in word_pos]

       I        - PRP
    omitted     - VBD
      the       - DT
  raspberries   - NNS
      and       - CC
     added      - VBD
 strawberries   - NNS
    instead     - RB


[None, None, None, None, None, None, None, None]

In [35]:
def get_pos(recept_id, recipes):
    words = word_tokenize(recipes[recipes.id == recept_id].name.values[0])
    
    row_pos = row_word = ''
    for word, pos in pos_tag(words):
        l = max(len(word), len(pos))
        row_pos += f"{pos:^{l}} "
        row_word += f"{word:^{l}} "
    
    print(row_pos)
    print(row_word)
    
get_pos(241106, recipes)
print()
get_pos(44123, recipes)

   JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS   
eggplant steaks with chickpeas feta cheese and black olives 

  NN   NN IN DT   NN   JJ    NN   NN  
george s  at the cove black bean soup 


In [36]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,"an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!",18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,"my children and their friends ask for my homemade popsicles morning, noon and night. i never turn them down; who am i to tell them that they are good for them! for variety i substitute different flavours of frozen juice - grape, fruit punch, tropical etc.",NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family get together. they are delicious!! a little messy to make but worth the effort!! have a helper and make an,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual dinner or wonderful for after the theatre snack served with a robust red wine. for dinner serve with rice & a small salad. almond rice pilaf is a great accompaniment (recipe posted separately ) to cook the meat you must first heat your oil - i do this by heating it to almost boiling on the stove and then transfering it to your fondue burner. buy good quality meat i recommend only using a fillet. have at least 3 sauces. if you want to serve 4-6 people just increase the meat to 2 lbs there will be enough sauce.these sauce recipes came from,NaN
